In [1]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import (
    OrdinalEncoder,
    StandardScaler,
)
from scipy.stats import loguniform

In [3]:
numeric_features = ['n_words']
text_feature = 'Text'
ordinal_features = ['sentiment']
drop_features = ['Id', 'Author']
target = 'Rating'

In [4]:
train_df = pd.read_csv("../Data/processed/train.csv")
X_train, y_train = train_df.drop(columns=[target] + drop_features), train_df[target]

In [5]:
 preprocessor = ColumnTransformer(
        transformers=[
            ('text', CountVectorizer(max_features=20_000, max_df=0.6), text_feature),
            ('num', StandardScaler(), numeric_features),
            ('ord', OrdinalEncoder(categories=[['neg', 'compound', 'neu', 'pos']]), ordinal_features)
        ]
    )
ml_pipe = Pipeline(
        steps=[
            ("prepro", preprocessor),
            ("Ridge", Ridge())
        ]
    )
ml_pipe

Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('text',
                                                  CountVectorizer(max_df=0.6,
                                                                  max_features=20000),
                                                  'Text'),
                                                 ('num', StandardScaler(),
                                                  ['n_words']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['neg',
                                                                              'compound',
                                                                              'neu',
                                                                              'pos']]),
                                                  ['sentiment'])])),
                ('Ridge', Ridge())])

In [63]:
 param_grid = {
        'Ridge__alpha': np.arange(800, 1200, 50)
    }

In [64]:
hyper_parameters_search = GridSearchCV(ml_pipe, param_grid=param_grid, n_jobs=-1, scoring='r2', verbose=1)

In [65]:
hyper_parameters_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   33.4s finished


GridSearchCV(estimator=Pipeline(steps=[('prepro',
                                        ColumnTransformer(transformers=[('text',
                                                                         CountVectorizer(max_df=0.6,
                                                                                         max_features=20000),
                                                                         'Text'),
                                                                        ('num',
                                                                         StandardScaler(),
                                                                         ['n_words']),
                                                                        ('ord',
                                                                         OrdinalEncoder(categories=[['neg',
                                                                                                     'compound',
                 

In [66]:
hyper_parameters_search.best_estimator_

Pipeline(steps=[('prepro',
                 ColumnTransformer(transformers=[('text',
                                                  CountVectorizer(max_df=0.6,
                                                                  max_features=20000),
                                                  'Text'),
                                                 ('num', StandardScaler(),
                                                  ['n_words']),
                                                 ('ord',
                                                  OrdinalEncoder(categories=[['neg',
                                                                              'compound',
                                                                              'neu',
                                                                              'pos']]),
                                                  ['sentiment'])])),
                ('Ridge', Ridge(alpha=800))])

In [67]:
results_df = {}
scores = cross_validate(
    hyper_parameters_search.best_estimator_,
    X_train,
    y_train,
    scoring='r2',
    return_train_score=True)

df = pd.DataFrame(scores)
results_df["Ridge"] = df.mean()
pd.DataFrame(results_df).T

,fit_time,score_time,test_score,train_score
Ridge,1.829247,0.438254,0.53279,0.799778
